In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

def dia():
    from datetime import date, timedelta
    dia = date.today() + timedelta(0)
    return dia

def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df
    
def grafico(df, nome):
    df = df.reset_index(drop=True)
    df.index += 1
    df['Profit_acu'] = df.Profit.cumsum()
    profit = round(df.Profit_acu.tail(1).item(),2)
    ROI = round((df.Profit_acu.tail(1)/len(df)*100).item(),2)
    df.Profit_acu.plot(title=nome, xlabel='Entradas', ylabel='Stakes')
    print("Profit:",profit,"stakes em", len(df),"jogos")
    print("ROI:",ROI,"%")

## Instalando as Dependências do Ubuntu

In [2]:
# %%shell
# # Ubuntu no longer distributes chromium-browser outside of snap
# #
# # Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# # Add debian buster
# cat > /etc/apt/sources.list.d/debian.list <<'EOF'
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
# EOF

# # Add keys
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

# apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
# apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
# apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# # Prefer debian repo for chromium* packages only
# # Note the double-blank lines between entries
# cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
# Package: *
# Pin: release a=eoan
# Pin-Priority: 500

# Package: *
# Pin: origin "deb.debian.org"
# Pin-Priority: 300

# Package: chromium*
# Pin: origin "deb.debian.org"
# Pin-Priority: 700
# EOF

# # Install chromium and chromium-driver
# apt-get update
# apt-get install chromium chromium-driver

# # Install selenium
# pip install selenium

## Configuração do Web-Driver

In [3]:
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
#options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.headless = True

from selenium.webdriver.chrome.service import Service
wd_Chrome = webdriver.Chrome("/usr/bin/chromedriver", options=options)

## Iniciando a Raspagem de Dados

In [4]:
# Com o WebDrive a gente consegue a pedir a página (URL)
wd_Chrome.get("https://www.flashscore.com/")

In [5]:
# Fechando Botão de Cookies
try:
    button_cookies = wd_Chrome.find_element(By.CSS_SELECTOR,'button#onetrust-accept-btn-handler')
    button_cookies.click()
except:
    pass

# Selecionando o Dia de Amanhã
wd_Chrome.find_element(By.CSS_SELECTOR,'button.calendar__navigation--tomorrow').click()
time.sleep(3)

# Pegando o ID dos Jogos
id_jogos = []
jogos = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.event__match--scheduled')

for i in jogos:
    id_jogos.append(i.get_attribute("id"))

# Exemplo de ID de um jogo: 'g_1_Gb7buXVt'    
id_jogos = [i[4:] for i in id_jogos]

jogo = {'Date':[],'Time':[],'Country':[],'League':[],'Home':[],'Away':[],'Odd_H':[],'Odd_D':[],'Odd_A':[],'Odd_Over15':[],'Odd_Under15':[],'Odd_Over25':[],'Odd_Under25':[],'Odd_BTTS_Yes':[],'Odd_BTTS_No':[]}

In [6]:
# Pegando as Informacoes Básicas do Jogo
for link in tqdm(id_jogos, total=len(id_jogos)):
    wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/match-summary')
    
    try:
        Date = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[0]
        Time = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[1]
        Country = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country').text.split(':')[0]
        League = wd_Chrome.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country')
        League = League.find_element(By.CSS_SELECTOR,'a').text
        Home = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__home')
        Home = Home.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
        Away = wd_Chrome.find_element(By.CSS_SELECTOR,'div.duelParticipant__away')
        Away = Away.find_element(By.CSS_SELECTOR,'div.participant__participantName').text
    except:
        pass

# Match Odds
    try:
        wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/1x2-odds/full-time')
        time.sleep(2)
        
        linhas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for linha in linhas:
                Odd_H = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                Odd_D = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text) 
                Odd_A = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[2].text)
    except:
        pass

# Over e Under 2.5
    try:
        wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/over-under/full-time')
        time.sleep(2)
        
        linhas_over_under = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table.oddsCell__odds')
        for i in linhas_over_under:
            linhas = i.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
            for linha in linhas:
                try:
                    qtd_gols = linha.find_element(By.CSS_SELECTOR,'span.oddsCell__noOddsCell').text.replace('.','')
                    if qtd_gols == '05':
                        Odd_Over05 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                        Odd_Under05 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
                    if qtd_gols == '15':
                        Odd_Over15 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                        Odd_Under15 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
                        #print(Odd_Under15)
                    if qtd_gols == '25':
                        Odd_Over25 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                        Odd_Under25 = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
                except:
                    pass
    except:
        pass

# Ambas Marcam (BTTS)
    try:
        wd_Chrome.get(f'https://www.flashscore.com/match/{link}/#/odds-comparison/both-teams-to-score/full-time')
        time.sleep(2)
    
        linhas = wd_Chrome.find_elements(By.CSS_SELECTOR,'div.ui-table__row')
        
        for linha in linhas:
                BTTS_Yes = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
                BTTS_No = float(linha.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
    except:
        pass    
    
    try:
        print(Date,Time,Country,League,Home,Away,Odd_H,Odd_D,Odd_A,Odd_Over15,Odd_Under15,Odd_Over25,Odd_Under25,BTTS_Yes,BTTS_No) 

        jogo['Date'].append(Date)
        jogo['Time'].append(Time)
        jogo['Country'].append(Country)
        jogo['League'].append(League)
        jogo['Home'].append(Home)
        jogo['Away'].append(Away)
        jogo['Odd_H'].append(Odd_H)
        jogo['Odd_D'].append(Odd_D)
        jogo['Odd_A'].append(Odd_A)
        jogo['Odd_Over15'].append(Odd_Over15)
        jogo['Odd_Under15'].append(Odd_Under15)
        jogo['Odd_Over25'].append(Odd_Over25)
        jogo['Odd_Under25'].append(Odd_Under25)
        jogo['Odd_BTTS_Yes'].append(BTTS_Yes)
        jogo['Odd_BTTS_No'].append(BTTS_No)

    except:
        pass

  0%|          | 0/234 [00:00<?, ?it/s]

  0%|          | 1/234 [00:09<37:02,  9.54s/it]

16.07.2023 20:30 NORTH & CENTRAL AMERICA GOLD CUP - PLAY OFFS - FINAL Mexico Panama 1.66 3.6 5.5 1.42 2.8 2.3 1.6 2.25 1.62


  1%|          | 2/234 [00:17<34:10,  8.84s/it]

16.07.2023 10:00 AFRICA COSAFA CUP - PLAY OFFS - 3RD PLACE Malawi South Africa 4.0 3.25 1.85 1.4 2.75 2.3 1.55 2.05 1.68


...

16.07.2023 10:00 ZIMBABWE PREMIER SOCCER LEAGUE - ROUND 15 Black Rhinos Hwange 2.6 3.4 2.5 1.28 3.4 1.87 1.87 1.7 2.1


100%|█████████▉| 233/234 [31:20<00:07,  7.36s/it]

16.07.2023 10:00 ZIMBABWE PREMIER SOCCER LEAGUE - ROUND 15 Herentals Highlanders 2.6 3.4 2.5 1.28 3.4 1.87 1.87 1.7 2.1


100%|██████████| 234/234 [31:27<00:00,  8.07s/it]

16.07.2023 10:00 ZIMBABWE PREMIER SOCCER LEAGUE - ROUND 15 Triangle Bulawayo Chiefs 2.6 3.4 2.5 1.28 3.4 1.87 1.87 1.7 2.1


## Gerando DataFrame com os Jogos do Dia

In [7]:
df = pd.DataFrame(jogo)
df = drop_reset_index(df)
df

,Date,Time,Country,League,Home,Away,Odd_H,Odd_D,Odd_A,Odd_Over15,Odd_Under15,Odd_Over25,Odd_Under25,Odd_BTTS_Yes,Odd_BTTS_No
1,16.07.2023,20:30,NORTH & CENTRAL AMERICA,GOLD CUP - PLAY OFFS - FINAL,Mexico,Panama,1.66,3.60,5.50,1.42,2.80,2.30,1.60,2.25,1.62
2,16.07.2023,10:00,AFRICA,COSAFA CUP - PLAY OFFS - 3RD PLACE,Malawi,South Africa,4.00,3.25,1.85,1.40,2.75,2.30,1.55,2.05,1.68
3,16.07.2023,13:00,AFRICA,COSAFA CUP - PLAY OFFS - FINAL,Lesotho,Zambia,4.75,3.50,1.66,1.36,2.88,2.10,1.66,2.05,1.70
4,16.07.2023,11:00,AFRICA,OLYMPIC GAMES WOMEN - QUALIFICATION - 1/8-FINALS,Rwanda W,Uganda W,4.75,3.50,1.66,1.36,2.88,2.10,1.66,2.05,1.70
5,16.07.2023,14:00,ARGENTINA,LIGA PROFESIONAL - ROUND 25,Belgrano,San Lorenzo,2.02,2.95,4.40,1.66,2.17,3.04,1.36,2.51,1.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,16.07.2023,19:00,WORLD,FRIENDLY INTERNATIONAL WOMEN,Puerto Rico W,Dominican Republic W,1.93,3.40,3.47,1.27,3.40,1.85,1.85,10.00,1.04
231,16.07.2023,23:00,WORLD,FRIENDLY INTERNATIONAL WOMEN,Colombia W,China W,2.60,3.40,2.50,1.28,3.40,1.87,1.87,1.70,2.10
232,16.07.2023,10:00,ZIMBABWE,PREMIER SOCCER LEAGUE - ROUND 15,Black Rhinos,Hwange,2.60,3.40,2.50,1.28,3.40,1.87,1.87,1.70,2.10
233,16.07.2023,10:00,ZIMBABWE,PREMIER SOCCER LEAGUE - ROUND 15,Herentals,Highlanders,2.60,3.40,2.50,1.28,3.40,1.87,1.87,1.70,2.10


In [8]:
df = df.rename(columns={'Odd_H': 'FT_Odd_H',
                        'Odd_D': 'FT_Odd_D',
                        'Odd_A': 'FT_Odd_A',
                        'Odd_Over15': 'FT_Odd_Over15',
                        'Odd_Under15': 'FT_Odd_Under15',
                        'Odd_Over25': 'FT_Odd_Over25',
                        'Odd_Under25': 'FT_Odd_Under25',
                        'Odd_BTTS_Yes': 'FT_Odd_BTTS_Yes',
                        'Odd_BTTS_No': 'FT_Odd_BTTS_No',
                        })

In [9]:
# Gerando os dados para excel
from datetime import date, datetime, timedelta
data = datetime.today() + timedelta(1)
amanha = data.strftime('%Y-%m-%d')
nome = 'Jogos_do_Dia_FlashScore.xlsx'

df.to_excel(f'G:\\Meu Drive\\Programacao\\futPythonTrader\\Jogos do dia\\{amanha}_{nome}', index = False)